In [10]:
#from libs.corpus import openConllu, check_projectivity
import pyconll
import pyconll.util
import torch
from transformers import AutoModel, AutoTokenizer
import numpy as np
from torch.nn.utils.rnn import pack_sequence
from torch.nn.utils.rnn import unpack_sequence

In [11]:

train = pyconll.load_from_file('data/it_isdt-ud-dev.conllu')
train_prepocesed=[]
for i,sent in enumerate(train):        
    sentence_preprocesed=[]
    for j,token in enumerate(sent):
        if(token.head is not None):
            sentence_preprocesed.append(token)
    train_prepocesed.append(sentence_preprocesed)

for i,sent in enumerate(train_prepocesed):     
    for j,token in enumerate(sent):
        print(token.head)
    print("------")

0
1
4
1
4
7
1
7
------
2
6
6
5
6
0
8
6
11
11
6
13
11
6
------
3
3
5
3
0
9
9
9
5
11
9
13
11
5
------
4
4
4
9
9
5
9
7
0
9
12
10
19
19
14
19
19
19
12
21
19
21
24
22
24
9
------
2
0
2
5
3
8
8
5
2
------
2
0
2
7
7
7
2
2
------
2
3
0
6
6
3
3
------
0
4
4
1
1
------
3
3
0
6
6
3
8
6
3
3
------
2
13
2
3
9
9
9
9
3
9
2
13
0
16
16
13
19
19
16
19
22
19
22
22
22
28
28
25
32
32
32
28
35
35
32
32
13
------
2
9
2
2
7
7
9
7
0
11
9
14
14
11
9
17
9
19
17
21
17
23
21
9
------
2
9
2
7
7
7
2
9
0
13
13
13
9
15
9
18
18
15
21
21
18
26
26
26
26
18
26
38
30
38
35
35
35
35
30
35
38
26
41
41
38
41
9
------
18
1
5
5
18
5
8
18
8
13
13
13
5
13
13
18
18
0
20
18
20
23
18
27
27
27
23
29
27
31
29
33
27
36
36
33
39
39
36
41
39
41
44
39
44
48
48
39
48
51
48
51
51
48
56
48
59
59
56
18
------
10
1
1
5
1
5
1
10
10
0
10
13
11
15
10
15
18
15
21
21
18
24
24
18
15
27
15
30
30
27
30
34
34
30
34
40
40
40
40
15
43
43
40
43
44
48
48
40
51
51
48
55
55
55
51
55
58
55
58
62
62
58
64
62
64
64
58
70
70
58
72
70
10
------
2
8
2
6
6
8
6
0
8


In [27]:

from torch import nn



#token_id = tokenizer.convert_tokens_to_ids(token_to_get_id)
#Trasformare l'array di parole in array di ID con la funzione soppra (se con le parole da UNK allora provare con i lemmi)
#creare padded_input_ids
#eseguire il modello con model() e torch 
#implementare l'algorimo di machine learning sugli embedding trovati e sulle varie features(capire come combinare le due)


#creiamo un oggetto Dependencies per salvare le depndencies
class Dependencies(object):
    def __init__(self,n):
        self.n = n
        self.heads = [None] * (n+1)
        self.arcs = []
    
    def add_arc(self, head, child):
        child=child
        self.heads[child]=head
        self.arcs.append((head,child))

    def contains(self,head,child):
        child=child
        if self.heads[child]==head:
            return True
        else: return False

        



class Oracle(object):
    
    def __init__(self) -> None:
        encoder_name = "dbmdz/bert-base-italian-xxl-cased"
        self.tokenizer = AutoTokenizer.from_pretrained(encoder_name)
        self.encoder = AutoModel.from_pretrained(encoder_name)
    
    def tokens(self,words,lemmas):
        words_token=[102]
        for i in range(0,len(lemmas)):
            word_token=self.tokenizer.convert_tokens_to_ids(words[i])
            lemma_token=self.tokenizer.convert_tokens_to_ids(lemmas[i])
            if(word_token!=101):
                words_token.append(word_token)
            else:
                words_token.append(lemma_token)
        words_token.append(103)
        
        return words_token
    
    def encode(self,words,lemmas):
        words_token=self.tokens(words,lemmas)
        #max_length = 64  # Example max length ????
        padded_input_ids = words_token
        input_tensor = torch.tensor([padded_input_ids])
        with torch.no_grad():
            outputs = self.encoder(input_tensor)
        return outputs.last_hidden_state




    def score(features):
        return None
        




#0:SHIFT 1:RIGHTARC 2:LEFTARC
moves = [0, 1, 2]

class Parser(object):

    model=Oracle()
        
    def transition(self,move, stack, i, dependencies):
        match move:
            case 0:
                stack.append(i)
                return stack,i+1,dependencies
            case 1:
                dependencies.add_arc(stack[-2], stack.pop())
                return stack,i,dependencies
            case 2:
                dependencies.add_arc(stack[-1], stack[-2])
                stack.pop(-2)
                return stack,i,dependencies
            case _:
                raise "Wrong Move"
            
    def get_valid_moves(i, n, stack_depth):
        moves = []
        if i <= n:
            moves.append(0)
        if stack_depth >= 2:
            moves.append(1)
            moves.append(2)
        return moves

    def parsing(self,words,tags):
        if(len(words)!=len(tags)):
            raise "Wrong tag len"
        n=len(words)
        deps=Dependencies(n)
        stack=[0]
        i_buffer=1
        moves=self.get_valid_moves(i_buffer,n,len(stack))
        while moves:
            features = ['FeaturesDaScegliere']
            scores = self.model.score(features)
            next_move = max(moves, key=lambda move: scores[move])
            stack,i_buffer,deps = self.transition(next_move, stack, i_buffer, deps)#Is stack being update?
            moves = self.get_valid_moves(i,n,len(stack))
        return deps
    

    def check_best(self,heads,stack,buffer,deps,i):
        move=-1
        if(len(stack)>=2):
            children_list=[]
            for child,head in enumerate(heads):
                if head == stack[-1]:
                    children_list.append(child)
            if(((heads[stack[-1]])==(stack[-2])) and all([deps.contains(stack[-1],child) for child in children_list])):
                move=1
            if(heads[stack[-2]]==stack[-1]):
                move=2
        if(i<=len(buffer) and move==-1):
            move=0
        elif(i>len(buffer) and move==-1):
            move=None
        return move
    

    def simulate_parse(self,heads,buffer):
        deps=Dependencies(len(buffer))
        stack=[0]
        moves=[]
        buffers=[]
        stacks=[]
        i=1
        best_move=self.check_best(heads,stack,buffer,deps,i)
        while best_move!=None:
            buffers.append(i)
            stacks.append(stack[:])
            moves.append(best_move)
            stack,i,deps=self.transition(best_move,stack,i,deps)
            best_move=self.check_best(heads,stack,buffer,deps,i)
        if(i>len(buffer)):
            return stacks,buffers,moves
        else: return None 

    def get_stack_context(self,list):
        depth=len(list)

        if depth >= 3:
            return [list[-1], list[-2], list[-3]]
        
        elif depth >= 2:
            
            return [list[-1], list[-2], -1]
        
        elif depth == 1:
            return [list[-1], -1 , -1]
        else:
            return [-1, -1, -1]

    def get_buffer_context(self,index,len_phrase):
        if(index==len_phrase):
            return [index,-1]
        elif(index>len_phrase):
            return [-1,-1]
        else: return [index,index+1]  

    def flatten_embedded_features(self,matrix):
        flat_list = torch.tensor([])
        for row in matrix:
            flat_list=torch.cat((flat_list,row))
        return flat_list

    def encode_moves(self,heads,phrase,phrases_lemma):
        embeddings = self.model.encode(phrase,phrases_lemma)
        stacks,buffers,moves=self.simulate_parse(heads,phrase)
        root_embeding=torch.tensor(np.ones(768))
        empty_embedding=torch.tensor(np.zeros(768))
        embedded_features=[]
        for i in range(0,len(stacks)):
            stack_feature=self.get_stack_context(stacks[i])
            for j,el in enumerate(stack_feature):
                if(el>=1):
                    stack_feature[j]=embeddings[0][el]
                elif(el==0):
                    stack_feature[j]=root_embeding
                else:
                    stack_feature[j]=empty_embedding
            buffer_feature=self.get_buffer_context(buffers[i],len(phrase))
            for j,el in enumerate(buffer_feature):
                if(el>=1):
                    buffer_feature[j]=embeddings[0][el]
                else:
                    buffer_feature[j]=empty_embedding

            embedded_features.append(self.flatten_embedded_features(stack_feature+buffer_feature).to(torch.float16))#.to(torch.float16)
        return embedded_features,moves

#DA FARE:
#1. Salvare batch da 50-100 frasi su file dati, ossia per ogni frase gli stati con relativi stack,buffer e move. 
#2. Importare batch per batch come nell'esempio di chat gpt.
#3. Per ogni batch fare forward e back-prop di adam optimizer come nell'esempio di chat_gpt.


parser = Parser()  
phrase = "book me the morning flight".split()
phrase2 = "Book the flight Houston".split()
heads=[-1,0,1,5,5,1]
heads2=[-1,0,3,1,3]
stack=[0]
parser= Parser()
sstack,buffer,moves= parser.simulate_parse(heads,phrase)
sstack2,buffer2,moves2= parser.simulate_parse(heads2,phrase2)


p1,m1=parser.encode_moves(heads,phrase,phrase)
p2,m2=parser.encode_moves(heads2,phrase2,phrase2)
torch.set_printoptions(profile="full")
p1 = torch.stack(p1, dim=0)
p2 = torch.stack(p2, dim=0)
#print(p1)
#print(p2)
input=pack_sequence([p1, p2])

input_size = 3840  # Each element in the sequence is a vector of size 2
hidden_size = 64
num_layers = 1
output_size = 3  # Example output size

model = torch.nn.LSTM(input_size,hidden_size,num_layers,batch_first=True,bidirectional=False,proj_size=output_size)
model.half()

output = model(input)
print("OUTPUT")
print(unpack_sequence(output[0]))





OUTPUT
[tensor([[-0.2869, -0.1644,  0.1593],
        [ 0.0380,  0.0155,  0.0022],
        [ 0.0043, -0.0454, -0.0872],
        [ 0.1622,  0.0690, -0.1196],
        [-0.0178,  0.2732, -0.2520],
        [-0.1400, -0.0475, -0.0512],
        [-0.1592,  0.3274, -0.0680],
        [-0.0715,  0.0823, -0.0699],
        [ 0.1316,  0.1544, -0.0072],
        [ 0.0729, -0.0393,  0.0637]], dtype=torch.float16,
       grad_fn=<IndexBackward0>), tensor([[-0.0270,  0.0183,  0.1122],
        [ 0.1235,  0.0618, -0.0569],
        [-0.0692,  0.1942, -0.0517],
        [-0.0479, -0.0295,  0.0131],
        [ 0.0151,  0.1785, -0.0876],
        [ 0.0873,  0.0539,  0.0312],
        [ 0.0654,  0.1396, -0.0700],
        [ 0.0097,  0.1877, -0.0973]], dtype=torch.float16,
       grad_fn=<IndexBackward0>)]
